In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.externals import joblib

# from sklearn.impute import SimpleImputer

In [2]:
world_cups = pd.read_csv('C:\Sushrut\Fifa world cup 2018\WorldCups.csv')
world_cup_player = pd.read_csv('C:\Sushrut\Fifa world cup 2018\WorldCupPlayers.csv')
world_cups_matches = pd.read_csv('C:\Sushrut\Fifa world cup 2018\WorldCupMatches.csv')

In [3]:
world_cup_player = world_cup_player.dropna()
world_cups_matches = world_cups_matches.dropna()
world_cup_player.tail()

,RoundID,MatchID,Team Initials,Coach Name,Line-up,Shirt Number,Player Name,Position,Event
37658,255955,300186490,NED,Louis VAN GAAL (NED),S,9,V. PERSIE,C,O96'
37693,255957,300186502,NED,Louis VAN GAAL (NED),S,1,CILLESSEN,GK,O90'
37694,255957,300186502,BRA,SCOLARI Luiz Felipe (BRA),S,3,T SILVA,C,Y2'
37705,255957,300186502,NED,Louis VAN GAAL (NED),S,9,V. PERSIE,C,P3'
37715,255957,300186502,NED,Louis VAN GAAL (NED),N,22,VORM,GK,I90'


In [4]:
def replace_name(df):
    if(df['Home Team Name'] in ['German DR', 'Germany FR']):
        df['Home Team Name'] = 'Germany'
    elif(df['Home Team Name'] == 'Soviet Union'):
        df['Home Team Name'] = 'Russia'
    
    if(df['Away Team Name'] in ['German DR', 'Germany FR']):
        df['Away Team Name'] = 'Germany'
    elif(df['Away Team Name'] == 'Soviet Union'):
        df['Away Team Name'] = 'Russia'
    return df
    
world_cups_matches = world_cups_matches.apply(replace_name, axis='columns')
world_cups_matches

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA
5,1930.0,16 Jul 1930 - 14:45,Group 1,Parque Central,Montevideo,Chile,3.0,0.0,Mexico,,9249.0,1.0,0.0,CRISTOPHE Henry (BEL),APHESTEGUY Martin (URU),LANGENUS Jean (BEL),201.0,1095.0,CHI,MEX
6,1930.0,17 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,4.0,0.0,Bolivia,,18306.0,0.0,0.0,MATEUCCI Francisco (URU),LOMBARDI Domingo (URU),WARNKEN Alberto (CHI),201.0,1092.0,YUG,BOL
7,1930.0,17 Jul 1930 - 14:45,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Paraguay,,18306.0,2.0,0.0,MACIAS Jose (ARG),APHESTEGUY Martin (URU),TEJADA Anibal (URU),201.0,1097.0,USA,PAR
8,1930.0,18 Jul 1930 - 14:30,Group 3,Estadio Centenario,Montevideo,Uruguay,1.0,0.0,Peru,,57735.0,0.0,0.0,LANGENUS Jean (BEL),BALWAY Thomas (FRA),CRISTOPHE Henry (BEL),201.0,1099.0,URU,PER
9,1930.0,19 Jul 1930 - 12:50,Group 1,Estadio Centenario,Montevideo,Chile,1.0,0.0,France,,2000.0,0.0,0.0,TEJADA Anibal (URU),LOMBARDI Domingo (URU),REGO Gilberto (BRA),201.0,1094.0,CHI,FRA


In [5]:
team_name = {}
index = 0

for idx, row in world_cups_matches.iterrows():
    name = row['Home Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
    name = row['Away Team Name']
    if(name not in team_name.keys()):
        team_name[name] = index
        index += 1
        
team_name

{'France': 0,
 'Mexico': 1,
 'USA': 2,
 'Belgium': 3,
 'Yugoslavia': 4,
 'Brazil': 5,
 'Romania': 6,
 'Peru': 7,
 'Argentina': 8,
 'Chile': 9,
 'Bolivia': 10,
 'Paraguay': 11,
 'Uruguay': 12,
 'Austria': 13,
 'Hungary': 14,
 'Egypt': 15,
 'Switzerland': 16,
 'Netherlands': 17,
 'Sweden': 18,
 'Germany': 19,
 'Spain': 20,
 'Italy': 21,
 'Czechoslovakia': 22,
 'Dutch East Indies': 23,
 'Cuba': 24,
 'Norway': 25,
 'Poland': 26,
 'England': 27,
 'Scotland': 28,
 'Turkey': 29,
 'Korea Republic': 30,
 'Russia': 31,
 'Wales': 32,
 'Northern Ireland': 33,
 'Colombia': 34,
 'Bulgaria': 35,
 'Korea DPR': 36,
 'Portugal': 37,
 'Israel': 38,
 'Morocco': 39,
 'El Salvador': 40,
 'Australia': 41,
 'Zaire': 42,
 'Haiti': 43,
 'Tunisia': 44,
 'IR Iran': 45,
 'Iran': 46,
 'Cameroon': 47,
 'New Zealand': 48,
 'Algeria': 49,
 'Honduras': 50,
 'Kuwait': 51,
 'Canada': 52,
 'Iraq': 53,
 'Denmark': 54,
 'rn">United Arab Emirates': 55,
 'Costa Rica': 56,
 'rn">Republic of Ireland': 57,
 'Saudi Arabia': 58,
 

In [6]:
# drop unnecessary columns
dropped_matches = world_cups_matches.drop(['Datetime', 'Stadium', 'Referee', 'Assistant 1', 'Assistant 2', 
                                           'RoundID','Win conditions','Home Team Initials', 'Away Team Initials', 
                                           'Half-time Home Goals', 'Half-time Away Goals', 'Attendance', 'City', 
                                           'MatchID', 'Stage'], 1)

In [7]:
championships = world_cups['Winner'].map(lambda p: 'Germany' if p=='Germany FR' else p).value_counts()
championships

Brazil       5
Germany      4
Italy        4
Uruguay      2
Argentina    2
France       1
Spain        1
England      1
Name: Winner, dtype: int64

In [8]:
dropped_matches['Home Team Championship'] = 0
dropped_matches['Away Team Championship'] = 0
dropped_matches.head()

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Home Team Championship,Away Team Championship
0,1930.0,France,4.0,1.0,Mexico,0,0
1,1930.0,USA,3.0,0.0,Belgium,0,0
2,1930.0,Yugoslavia,2.0,1.0,Brazil,0,0
3,1930.0,Romania,3.0,1.0,Peru,0,0
4,1930.0,Argentina,1.0,0.0,France,0,0


In [9]:
def count_championship(df):
    if(championships.get(df['Home Team Name']) != None):
        df['Home Team Championship'] = championships.get(df['Home Team Name'])
    if(championships.get(df['Away Team Name']) != None):
        df['Away Team Championship'] = championships.get(df['Away Team Name'])
    return df

dropped_matches = dropped_matches.apply(count_championship, axis='columns')

dropped_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Home Team Championship,Away Team Championship
0,1930.0,France,4.0,1.0,Mexico,1,0
1,1930.0,USA,3.0,0.0,Belgium,0,0
2,1930.0,Yugoslavia,2.0,1.0,Brazil,0,5
3,1930.0,Romania,3.0,1.0,Peru,0,0
4,1930.0,Argentina,1.0,0.0,France,2,1
5,1930.0,Chile,3.0,0.0,Mexico,0,0
6,1930.0,Yugoslavia,4.0,0.0,Bolivia,0,0
7,1930.0,USA,3.0,0.0,Paraguay,0,0
8,1930.0,Uruguay,1.0,0.0,Peru,2,0
9,1930.0,Chile,1.0,0.0,France,0,1


In [10]:
#find who won: Home win: 1, Away win: 2, Draw: 0
dropped_matches['Winner'] = '-'

def find_winner(df):
    if(int(df['Home Team Goals']) == int(df['Away Team Goals'])):
        df['Winner'] = 0
    elif(int(df['Home Team Goals']) > int(df['Away Team Goals'])):
        df['Winner'] = 1
    else:
        df['Winner'] = 2
    return df

dropped_matches = dropped_matches.apply(find_winner, axis='columns')

dropped_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Home Team Championship,Away Team Championship,Winner
0,1930.0,France,4.0,1.0,Mexico,1,0,1
1,1930.0,USA,3.0,0.0,Belgium,0,0,1
2,1930.0,Yugoslavia,2.0,1.0,Brazil,0,5,1
3,1930.0,Romania,3.0,1.0,Peru,0,0,1
4,1930.0,Argentina,1.0,0.0,France,2,1,1
5,1930.0,Chile,3.0,0.0,Mexico,0,0,1
6,1930.0,Yugoslavia,4.0,0.0,Bolivia,0,0,1
7,1930.0,USA,3.0,0.0,Paraguay,0,0,1
8,1930.0,Uruguay,1.0,0.0,Peru,2,0,1
9,1930.0,Chile,1.0,0.0,France,0,1,1


In [11]:
#replace team name by id in team_name dictionary

def replace_team_name_by_id(df):
    df['Home Team Name'] = team_name[df['Home Team Name']]
    df['Away Team Name'] = team_name[df['Away Team Name']]
    #df['Winner'] = team_name[df['Winner']]
    return df

teamid_matches = dropped_matches.apply(replace_team_name_by_id, axis='columns')
teamid_matches

,Year,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Home Team Championship,Away Team Championship,Winner
0,1930.0,0,4.0,1.0,1,1,0,1
1,1930.0,2,3.0,0.0,3,0,0,1
2,1930.0,4,2.0,1.0,5,0,5,1
3,1930.0,6,3.0,1.0,7,0,0,1
4,1930.0,8,1.0,0.0,0,2,1,1
5,1930.0,9,3.0,0.0,1,0,0,1
6,1930.0,4,4.0,0.0,10,0,0,1
7,1930.0,2,3.0,0.0,11,0,0,1
8,1930.0,12,1.0,0.0,7,2,0,1
9,1930.0,9,1.0,0.0,0,0,1,1


In [12]:
#drop unecessary columns
teamid_matches = teamid_matches.drop(['Year', 'Home Team Goals', 'Away Team Goals'], 1)

teamid_matches

,Home Team Name,Away Team Name,Home Team Championship,Away Team Championship,Winner
0,0,1,1,0,1
1,2,3,0,0,1
2,4,5,0,5,1
3,6,7,0,0,1
4,8,0,2,1,1
5,9,1,0,0,1
6,4,10,0,0,1
7,2,11,0,0,1
8,12,7,2,0,1
9,9,0,0,1,1


In [13]:
X = teamid_matches.loc[:,['Home Team Name','Away Team Name','Home Team Championship','Away Team Championship']]
X = np.array(X).astype('float64')
X

array([[ 0.,  1.,  1.,  0.],
       [ 2.,  3.,  0.,  0.],
       [ 4.,  5.,  0.,  5.],
       ...,
       [17.,  8.,  0.,  2.],
       [ 5., 17.,  5.,  0.],
       [19.,  8.,  4.,  2.]])

In [14]:
_X = X.copy()

_X[:,0] = X[:,1]
_X[:,1] = X[:,0]
_X[:,2] = X[:,3]
_X[:,3] = X[:,2]

_X

array([[ 1.,  0.,  0.,  1.],
       [ 3.,  2.,  0.,  0.],
       [ 5.,  4.,  5.,  0.],
       ...,
       [ 8., 17.,  2.,  0.],
       [17.,  5.,  0.,  5.],
       [ 8., 19.,  2.,  4.]])

In [15]:
y = dropped_matches.loc[:,['Winner']]
y = np.array(y).astype('int')
y = np.reshape(y,(1,850))
y = y[0]

_y = y.copy()

for i in range(len(_y)):
  if(_y[i]==1):
    _y[i] = 2
  elif(_y[i] ==2):
    _y[i] = 1

In [16]:
X = np.concatenate((X,_X), axis= 0)

y = np.concatenate((y,_y))

In [17]:
X

array([[ 0.,  1.,  1.,  0.],
       [ 2.,  3.,  0.,  0.],
       [ 4.,  5.,  0.,  5.],
       ...,
       [ 8., 17.,  2.,  0.],
       [17.,  5.,  0.,  5.],
       [ 8., 19.,  2.,  4.]])

In [18]:
y

array([1, 1, 1, ..., 0, 1, 2])

In [19]:
#shuffle and split test, train
X,y = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(y_train)

[2 1 1 ... 1 1 2]


In [20]:
#train by svm
# param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
#               'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
param_grid = {'C': [1e3],
              'gamma': [0.0001] }
svm_model = SVC(kernel='rbf', class_weight='balanced', probability=True)
svm_model.fit(X, y)
#print("Best estimator found by grid search:")
#print(svm_model.best_estimator_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [21]:
print("Predicting on the test set")
#t0 = time()
y_pred = svm_model.predict(X_test)
#print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, labels=range(3)))

Predicting on the test set
              precision    recall  f1-score   support

           0       0.63      0.79      0.70        97
           1       0.78      0.76      0.77       166
           2       0.79      0.69      0.73       162

   micro avg       0.74      0.74      0.74       425
   macro avg       0.73      0.75      0.73       425
weighted avg       0.75      0.74      0.74       425

[[ 77  10  10]
 [ 20 126  20]
 [ 26  25 111]]


In [22]:
def prediction(team1, team2):
  id1 = team_name[team1]
  id2 = team_name[team2]
  championship1 = championships.get(team1) if championships.get(team1) != None else 0
  championship2 = championships.get(team2) if championships.get(team2) != None else 0

  x = np.array([id1, id2, championship1, championship2]).astype('float64')
  x = np.reshape(x, (1,-1))
  _y = svm_model.predict_proba(x)[0]

  text = ('Chance for '+team1+' to win '+team2+' is {}\nChance for '+team2+' to win '+team1+' is {}\nChance for '+team1+' and '+team2+' draw is {}').format(_y[1]*100,_y[2]*100,_y[0]*100)
  return _y[0], text

In [23]:
#predict match between France and Uruguay
prob1, text1 = prediction('France', 'Uruguay')
print(text1)

Chance for France to win Uruguay is 24.55905277917895
Chance for Uruguay to win France is 46.72942291862564
Chance for France and Uruguay draw is 28.7115243021954


In [24]:
#predict match between Brazil and Belgium
prob2, text2 = prediction('Brazil','Belgium')
print(text2)

Chance for Brazil to win Belgium is 53.19261583697745
Chance for Belgium to win Brazil is 21.679703845651808
Chance for Brazil and Belgium draw is 25.127680317370775


In [25]:
#predict match between Brazil and Italy
prob3, text3 = prediction('Brazil','Italy')
print(text3)

Chance for Brazil to win Italy is 40.41802262923948
Chance for Italy to win Brazil is 39.54639621064288
Chance for Brazil and Italy draw is 20.035581160117623
